<a href="https://colab.research.google.com/github/yuichi0625/cellar/blob/main/check_country_language_pair_on_transformers/bert_japanese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install fugashi
!pip install ipadic

     |████████████████████████████████| 2.0MB 5.5MB/s 
     |████████████████████████████████| 3.2MB 43.5MB/s 
     |████████████████████████████████| 890kB 48.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=672d975f178e9bfdf83b46aa34c8fb725e29d90510af5379e031a627353d91a1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 491kB 4.8MB/s 
     |████████████████████████████████| 13.4MB 497kB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-cp37-none-any.whl size=13556725 sha256=7792c2406335115401432639fdb7656243f240f7b0590bf4dd3d24e2c32ec9ae
  Stored in directory: /root/.cache/pip/wheels/ff/00/d1/0c094a0ce58a77199a0c5801f0ecf510c80f0ecbec27f07d2c
Successfully built ipadic


In [2]:
!wget https://raw.githubusercontent.com/yuichi0625/cellar/main/check_country_language_pair_on_transformers/countries.csv

--2021-03-23 16:22:07--  https://raw.githubusercontent.com/yuichi0625/cellar/main/check_country_language_pair_on_transformers/countries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7068 (6.9K) [text/plain]
Saving to: ‘countries.csv’

countries.csv       100%[===================>]   6.90K  --.-KB/s    in 0s      

2021-03-23 16:22:07 (90.2 MB/s) - ‘countries.csv’ saved [7068/7068]



In [3]:
import pandas as pd
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM
pd.set_option('display.max_rows', 500)

tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

sequence = '私は{}に住んでいて、話す言語は{}です。'

rows = []
for name in pd.read_csv('countries.csv')['ja'].sort_values():
  input = tokenizer.encode(sequence.format(name, tokenizer.mask_token), return_tensors='pt')
  mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

  token_logits = model(input).logits[0]
  mask_token_logits = token_logits[mask_token_index, :][0]
  mask_token_probs = torch.softmax(mask_token_logits, dim=0)

  candidate_indices = torch.topk(mask_token_logits, k=3, dim=0).indices.tolist()
  result = [f'{tokenizer.decode([ci])} ({mask_token_probs[ci] * 100:.0f}%)' for ci in candidate_indices]

  rows.append([name, *result])

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
pd.DataFrame(rows, columns=['国', '言語(top1)', '言語(top2)', '言語(top3)'])

,国,言語(top1),言語(top2),言語(top3)
0,アイスランド,英語 (70%),日本語 (10%),フランス語 (4%)
1,アイルランド,英語 (75%),フランス語 (20%),ラテン語 (1%)
2,アゼルバイジャン,英語 (59%),日本語 (12%),フランス語 (9%)
3,アフガニスタン,英語 (57%),日本語 (21%),フランス語 (4%)
4,アメリカ合衆国,英語 (76%),フランス語 (10%),日本語 (6%)
5,アラブ首長国連邦,英語 (50%),アラブ (22%),日本語 (6%)
6,アルジェリア,フランス語 (83%),英語 (10%),ラテン語 (2%)
7,アルゼンチン,英語 (44%),フランス語 (18%),日本語 (13%)
8,アルバニア,英語 (46%),フランス語 (26%),日本語 (9%)
9,アルメニア,英語 (37%),フランス語 (31%),日本語 (6%)
